In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers

In [16]:

class MyImageClassifier:
    def __init__(self):
        self.model = self.build_model()
        self.loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
        self.optimizer = tf.keras.optimizers.Adam()
        self.train_loss = tf.keras.metrics.Mean(name='train_loss')
        self.train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
        self.test_loss = tf.keras.metrics.Mean(name='test_loss')
        self.test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

    def build_model(self):
        model = models.Sequential()
        # Add your model architecture layers here
        # For example:
        model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Flatten())
        model.add(layers.Dense(64, activation='relu'))
        model.add(layers.Dense(2))  # Assuming 2 classes (male and female)

        return model

    @tf.function
    def train_step(self, images, labels):
        with tf.GradientTape() as tape:
            predictions = self.model(images, training=True)
            loss = self.loss_object(labels, predictions)
        gradients = tape.gradient(loss, self.model.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.model.trainable_variables))

        self.train_loss(loss)
        self.train_accuracy(labels, predictions)

    @tf.function
    def test_step(self, images, labels):
        predictions = self.model(images, training=False)
        t_loss = self.loss_object(labels, predictions)

        self.test_loss(t_loss)
        self.test_accuracy(labels, predictions)

    def train_model(self, train_dataset, test_dataset, epochs):
        for epoch in range(epochs):
            self.train_loss.reset_states()
            self.train_accuracy.reset_states()
            self.test_loss.reset_states()
            self.test_accuracy.reset_states()

            for images, labels in train_dataset:
                self.train_step(images, labels)

            for test_images, test_labels in test_dataset:
                self.test_step(test_images, test_labels)

            print(
                f'Epoch {epoch + 1}, '
                f'Loss: {self.train_loss.result()}, '
                f'Accuracy: {self.train_accuracy.result() * 100}, '
                f'Test Loss: {self.test_loss.result()}, '
                f'Test Accuracy: {self.test_accuracy.result() * 100}'
            )


In [21]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create an ImageDataGenerator with data augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Use the generator to load and augment the images
train_dataset_augmented = datagen.flow_from_directory(
    r'C:\Users\Hp\Downloads\VW_AI\dataset\train',
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'  # Change to 'categorical' if you have more than two classes
)


Found 8 images belonging to 2 classes.


In [22]:
train_path = r'C:\Users\Hp\Downloads\VW_AI\dataset\train'
test_path = r'C:\Users\Hp\Downloads\VW_AI\dataset\validation'

# Create train dataset
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_path,
    image_size=(128, 128),
    batch_size=32
)

# Create test dataset
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_path,
    image_size=(128, 128),
    batch_size=32
)

# Normalize pixel values to be between 0 and 1
train_dataset = train_dataset.map(lambda x, y: (x / 255.0, y))
test_dataset = test_dataset.map(lambda x, y: (x / 255.0, y))


Found 8 files belonging to 2 classes.
Found 8 files belonging to 2 classes.


In [23]:
# Instantiate the classifier
classifier = MyImageClassifier()

# Train the model
classifier.train_model(train_dataset, test_dataset, epochs=5)

Epoch 1, Loss: 0.7015434503555298, Accuracy: 50.0, Test Loss: 7.113512992858887, Test Accuracy: 50.0
Epoch 2, Loss: 7.132041931152344, Accuracy: 50.0, Test Loss: 8.265257835388184, Test Accuracy: 50.0
Epoch 3, Loss: 7.8975372314453125, Accuracy: 50.0, Test Loss: 8.492708206176758, Test Accuracy: 50.0
Epoch 4, Loss: 7.757011890411377, Accuracy: 50.0, Test Loss: 5.6743669509887695, Test Accuracy: 50.0
Epoch 5, Loss: 4.935980796813965, Accuracy: 50.0, Test Loss: 1.8645808696746826, Test Accuracy: 50.0
